In [2]:
#Import Statements 
import torch
import numpy as np
import transformers
from transformers import AutoTokenizer,RobertaTokenizer
from transformers import AutoConfig
from transformers import AutoModelForSeq2SeqLM
import datasets
from datasets import load_dataset, load_metric
# import evaluator
import evaluate
from transformers import EarlyStoppingCallback, IntervalStrategy

In [1]:
import wandb
import os

os.environ["WANDB_PROJECT"]="code_gen_codeT5_small_config"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

In [3]:
#Checking GPU 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [4]:
# Loading CoNaLa Dataset

data= load_dataset("AhmedSSoliman/CoNaLa-Large")


Using custom data configuration AhmedSSoliman--CoNaLa-Large-51e462b03558a775
Found cached dataset csv (C:/Users/Dharneeshkar J/.cache/huggingface/datasets/AhmedSSoliman___csv/AhmedSSoliman--CoNaLa-Large-51e462b03558a775/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
data

DatasetDict({
    train: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 24687
    })
    test: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['intent', 'snippet'],
        num_rows: 1237
    })
})

In [6]:
trainData= data["train"]
valData = data["validation"]
testData = data["test"]

In [7]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')
model.config.early_stopping = True

model.config.encoder_layerdrop = 0.2
model.config.decoder_layerdrop = 0.2

model.config.dropout = 0.2

In [8]:


# map data correctly
def map_to_encoder_decoder_inputs(batch):  
    
    inputs = tokenizer(str(batch["intent"][0]), return_tensors="pt",truncation=True, padding=True)
    outputs = tokenizer(str(batch["snippet"][0]), return_tensors="pt",truncation=True,padding=True)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids
   
    return batch

In [9]:
train_data = trainData.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet' ],
)


Parameter 'function'=<function map_to_encoder_decoder_inputs at 0x0000023AAF4C14C0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/24687 [00:00<?, ?ba/s]

In [10]:
val_data = valData.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet' ],
)


  0%|          | 0/1237 [00:00<?, ?ba/s]

In [11]:
import nltk
import CodeBLEU
# !pip install tree-sitter
# !pip install code_bert_score
nltk.download("punkt", quiet=True)
metric1 = evaluate.load("bleu")
metric2 = evaluate.load("neulab/python_bleu")
metric4 = evaluate.load("idsedykh/codebleu2")

import code_bert_score
# pred_results = code_bert_score.score(cands=predictions, refs=refs, lang='python')
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    m1 = metric1.compute(predictions=decoded_preds, references=decoded_labels)
    m2= metric2.compute(predictions=decoded_preds, references=decoded_labels)
    m3 = code_bert_score.score(cands=decoded_preds, refs=decoded_labels, lang='python')
#     m4= metric4.compute(predictions=decoded_preds, references=decoded_labels)
#     CodeBLEU.calc_code_bleu(hyp=decoded_preds , refs=decoded_labels , lang='python')
    return {
                'BLEU'                  : m1['bleu'] * 100,
                'BLEU-Unigram-Precision': 100 * m1['precisions'][0],
                'BLEU-Bigram-Precision' : 100 * m1['precisions'][1],
                'Python-BLEU' : m2['bleu_score'],
                'Code-BERT-SCORE-Precision' : m3[0].mean(),
                'Code-BERT-SCORE-Recall' : m3[1].mean(),
                'Code-BERT-SCORE-F1': m3[2].mean(),
              
              
            }
    

In [12]:
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True, model = model)

In [13]:
import ml_collections
def trainingArgs(logging_dir, label_names = None):
    if label_names is None:
        label_names = ['labels']
    _config = ml_collections.ConfigDict()
    _config.output_dir = logging_dir
    _config.evaluation_strategy = "epoch"
    _config.save_strategy = "epoch"
    _config.per_device_train_batch_size = 2
    _config.per_device_eval_batch_size =2
    _config.predict_with_generate=True
    _config.logging_dir = logging_dir
    _config.save_total_limit = 2
    _config.learning_rate = 5e-5
    _config.num_train_epochs = 30
    _config.dataloader_num_workers = 0
    _config.label_names = label_names
    _config.no_cuda = False
    _config.load_best_model_at_end = True
    _config.group_by_length =True
    _config.warmup_ratio = 0.05
    _config.optim = "adamw_torch"
   
    _config.logging_strategy='epoch'
    _config.report_to="wandb"
    return _config  

In [14]:
args_dict = trainingArgs("./Exp_final10_CodeT5-small+config/")
training_args = transformers.Seq2SeqTrainingArguments(**args_dict)
training_args.predict_with_generate= True

In [15]:
trainer = transformers.Seq2SeqTrainer(
    model=model,
    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics,
   
)

In [16]:
from transformers.utils import logging

logging.set_verbosity_info()
logger = logging.get_logger("transformers")
logger.info("INFO")
logger.warning("WARN")

INFO
WARN


In [17]:
trainer.train()

***** Running training *****
  Num examples = 24687
  Num Epochs = 30
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 370320
  Number of trainable parameters = 60492288
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: dharneeshkar. Use `wandb login --relogin` to force relogin


***** Running Evaluation *****
  Num examples = 1237
  Batch size = 2


{'loss': 2.3847, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}


loading file vocab.json from cache at C:\Users\Dharneeshkar J/.cache\huggingface\hub\models--neulab--codebert-python\snapshots\0375ce6e7f53d21a181fa3d7485b1cd9a9e8c13f\vocab.json
loading file merges.txt from cache at C:\Users\Dharneeshkar J/.cache\huggingface\hub\models--neulab--codebert-python\snapshots\0375ce6e7f53d21a181fa3d7485b1cd9a9e8c13f\merges.txt
loading file tokenizer.json from cache at C:\Users\Dharneeshkar J/.cache\huggingface\hub\models--neulab--codebert-python\snapshots\0375ce6e7f53d21a181fa3d7485b1cd9a9e8c13f\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\Dharneeshkar J/.cache\huggingface\hub\models--neulab--codebert-python\snapshots\0375ce6e7f53d21a181fa3d7485b1cd9a9e8c13f\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\Dharneeshkar J/.cache\huggingface\hub\models--neulab--codebert-python\snapshots\0375ce6e7f53d21a181fa3d7485b1cd9a9e8c13f\tokenizer_config.json

{'eval_loss': 1.6648130416870117, 'eval_BLEU': 15.474104561248597, 'eval_BLEU-Unigram-Precision': 47.843064336184455, 'eval_BLEU-Bigram-Precision': 22.481036450292006, 'eval_Python-BLEU': 0.16610400126155941, 'eval_Code-BERT-SCORE-Precision': 0.7828806042671204, 'eval_Code-BERT-SCORE-Recall': 0.771892249584198, 'eval_Code-BERT-SCORE-F1': 0.776421070098877, 'eval_runtime': 126.8995, 'eval_samples_per_second': 9.748, 'eval_steps_per_second': 4.878, 'epoch': 1.0}


...

{'eval_loss': 0.5305931568145752, 'eval_BLEU': 56.83375637789568, 'eval_BLEU-Unigram-Precision': 79.27430718647253, 'eval_BLEU-Bigram-Precision': 68.85723330167775, 'eval_Python-BLEU': 0.5741858665473539, 'eval_Code-BERT-SCORE-Precision': 0.9014031887054443, 'eval_Code-BERT-SCORE-Recall': 0.8878775835037231, 'eval_Code-BERT-SCORE-F1': 0.8939470648765564, 'eval_runtime': 135.5365, 'eval_samples_per_second': 9.127, 'eval_steps_per_second': 4.567, 'epoch': 30.0}


Model weights saved in ./Exp_final10_CodeT5-small+config/checkpoint-370320\pytorch_model.bin
tokenizer config file saved in ./Exp_final10_CodeT5-small+config/checkpoint-370320\tokenizer_config.json
Special tokens file saved in ./Exp_final10_CodeT5-small+config/checkpoint-370320\special_tokens_map.json
Deleting older checkpoint [Exp_final10_CodeT5-small+config\checkpoint-345632] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./Exp_final10_CodeT5-small+config/checkpoint-357976 (score: 0.5304309129714966).
Saving model checkpoint to C:\Users\DHARNE~1\AppData\Local\Temp\tmpi_hae8dg
Configuration saved in C:\Users\DHARNE~1\AppData\Local\Temp\tmpi_hae8dg\config.json


{'train_runtime': 30814.9849, 'train_samples_per_second': 24.034, 'train_steps_per_second': 12.018, 'train_loss': 0.44341127861650903, 'epoch': 30.0}


Model weights saved in C:\Users\DHARNE~1\AppData\Local\Temp\tmpi_hae8dg\pytorch_model.bin
tokenizer config file saved in C:\Users\DHARNE~1\AppData\Local\Temp\tmpi_hae8dg\tokenizer_config.json
Special tokens file saved in C:\Users\DHARNE~1\AppData\Local\Temp\tmpi_hae8dg\special_tokens_map.json


TrainOutput(global_step=370320, training_loss=0.44341127861650903, metrics={'train_runtime': 30814.9849, 'train_samples_per_second': 24.034, 'train_steps_per_second': 12.018, 'train_loss': 0.44341127861650903, 'epoch': 30.0})

In [20]:
trainer.save_model()
trainer.save_state()

Saving model checkpoint to ./Exp_final10_CodeT5-small+config/
Configuration saved in ./Exp_final10_CodeT5-small+config/config.json
Model weights saved in ./Exp_final10_CodeT5-small+config/pytorch_model.bin
tokenizer config file saved in ./Exp_final10_CodeT5-small+config/tokenizer_config.json
Special tokens file saved in ./Exp_final10_CodeT5-small+config/special_tokens_map.json


In [21]:
wandb.finish()

eval/BLEU,▁▂▂▃▄▅▅▆▆▆▆▆▇▇▇▇▇▇████████████
eval/BLEU-Bigram-Precision,▁▂▃▃▄▅▅▆▆▆▆▆▇▇▇▇▇▇████████████
eval/BLEU-Unigram-Precision,▁▂▃▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇███████████
eval/Code-BERT-SCORE-F1,▁▂▃▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇████████████
eval/Code-BERT-SCORE-Precision,▁▂▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇████████████
eval/Code-BERT-SCORE-Recall,▁▂▃▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇████████████
eval/Python-BLEU,▁▂▂▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇████████████
eval/loss,█▆▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▂▂█▂▃▂▂▂▂▁▁▁▁▁▁▁▁▃▃▄▃▂▂▂▄▂▂▃▂▃
eval/samples_per_second,▇▇▁▇▆▇▆▆▆████████▅▆▅▅▇▇▆▄▇▇▆▇▅
eval/steps_per_second,▇▇▁▇▆▇▆▆▆████████▅▆▅▅▇▇▆▄▇▇▆▇▅


In [19]:
pytorch_total_params = sum(p.numel() for p in model.parameters())